In [74]:
import numpy as np
from numpy import linalg
import math
import spiceypy
import matplotlib.pyplot as plt
import csv
mu = 398600 # [km^3/s^2] for earth

# Universal Variable Orbit Propagator
Based off of Curtis 3.3 and 3.4 algorithms

questions to ask:

### Helper Functions

- S and C Stumpff functions
- Gaussian functions and derivatives (f and g)

In [75]:
def stumpffS(z):
    if (z > 0):
        s = (np.sqrt(z) - np.sin(np.sqrt(z)))/(np.sqrt(z))**3
    elif (z < 0):
        s = (np.sinh(np.sqrt(-z)) - np.sqrt(-z))/(np.sqrt(-z))**3
    else:
        s = 1/6
    
    return s

In [76]:
def stumpffC(z):
    if (z > 0):
        c = (1 - np.cos(np.sqrt(z)))/z
    elif (z < 0):
        c = (np.cosh(np.sqrt(-z)) - 1)/(-z)
    else:
        c = 1/2
    
    return c

In [77]:
def f_and_g(chi, dt, r0, alpha):
    # given universal anomaly, elapsed time, initial radial position, 1/a
    
    mu = 398600 # km^3/s^2 for earth
    
    z = alpha*chi**2
    
    f = 1 - chi**2/r0*stumpffC(z)
    g = t - 1/np.sqrt(mu)*chi**3*stumpffS(z)
    
    return f, g

In [78]:
def f_and_gDot(chi, r, r0, alpha):
    
    z = alpha*chi**2
    
    fdot = np.sqrt(mu)/r/r0*(z*stumpffS(z)-1)*chi
    gdot = 1 - chi**2/r*stumpffC(z)
    
    return fdot, gdot

# Kepler Solver
- Universal Anomaly, $\chi$
- Calculate state vector after elapsed time $\Delta T$

In [79]:
def solve_universal_kepler(dt, r0, vr0, alpha):
    # given dt, r0, vr0, a, find chi
    
    error = 10**(-8) # error tolerance
    nMax = 10000
    
    chi0 = np.sqrt(mu)*abs(alpha)*dt
    
    ratio = 1
    chi = chi0
    
    while (abs(ratio) > error):
        z = alpha*chi**2
        s = stumpffS(z)
        c = stumpffC(z)
        f = r0*vr0/np.sqrt(mu)*chi**2*c + (1-alpha*r0)*chi**3*s + r0*chi - np.sqrt(mu)*dt
        f1 = r0*vr0/np.sqrt(mu)*chi*(1-alpha*chi**2*s) + (1-alpha*r0)*chi**2*c + r0
        ratio = f/f1
        chi = chi-ratio
    
    return chi     

In [80]:
def state_vec(r0, v0, dt):
    # given initial vectors and elapsed time, calculate the state vector
    
    # initial vector magnitudes
    r0_mag = np.linalg.norm(r0)
    v0_mag = np.linalg.norm(v0)
    
    vr0 = np.dot(r0,v0)/r0_mag # initial radial velocity
    
    alpha = 2/r0_mag - v0_mag**2/mu # reciprocal of the semimajor axis (energy eqn)
    
    chi = solve_universal_kepler(dt, r0_mag, vr0, alpha) # universal anomaly
    
    f, g = f_and_g(chi, dt, r0_mag, alpha) # gaussian functions
    
    # final position vector and magnitude
    R = f*r0 + g*v0 
    r = np.linalg.norm(R)
    
    fDot, gDot = f_and_gDot(chi, r, r0_mag, alpha) # gauss
    
    # final velocity vector
    V = fDot*r0 + gDot*v0
    
    return R, V

# Lambert Solver (Curtis 5.2)
- Use $p$ or $r$ to denote prograde or retrograde trajectory.
- Takes initial/final position vectors, elapsed time, prograde/retrograde, and returns initial/final velocity vectors.

In [ ]:
def lambert_Curtis(r1, r2, dt, grade):
    
    mu = 398600 
    
    r1_mag = np.linalg.norm(r1)
    r2_mag = np.linalg.norm(r2)
    cross_12 = np.cross(r1, r2)
    theta = math.acos(np.dot(r1,r2)/r1_mag/r2_mag)
    
    if (grade == "r"):
        if (cross_12[2] >= 0):
            theta = 2*np.pi - theta
    else: # assume prograde trajectory if user does not input either 'r' or 'p'
        if (cross_12[2] <= 0):
            theta = 2*np.pi - theta
    
    A = np.sin(theta)*np.sqrt(r1_mag*r2_mag/(1-np.cos(theta)))
    
    def y(z):
        y = r1_mag + r2_mag + A*(z*stumpffS(z) - 1)/np.sqrt(stumpffC(z))
        #print(A)
        #print("y:", y)
        return y
    
    def fzt(z,dt):
        s = stumpffS(z)
        c = stumpffC(z)
        yz = y(z)
        #print(s, c, yz)
        f = (yz/c)**(3/2)*s + A*np.sqrt(yz) - np.sqrt(mu)*dt
        #print((yz/c)**(3/2))
        #print("F:", f)
        return f
    
    def dfdz(z):
        if (z == 0):
            y0 = y(0)
            return np.sqrt(2)/40*y0**(3/2) + A/8*np.sqrt(y0) + A*np.sqrt(1/2/y0)
        else:
            s = stumpffS(z)
            c = stumpffC(z)
            yz = y(z)
            return (yz/c)**(3/2)*((1/2/z)*(c - (3*s)/(2*c)) + 3*s**2/4/c) \
        + A/8*(3*s/c*np.sqrt(yz) + A*np.sqrt(c/yz))
    
    z = 0
    while (fzt(z,dt) < 0):
        #print(fzt(z,dt))
        z = z + 0.1
        
    tol = 1e-8
    
    ratio = 1
    n = 0
    while ((abs(ratio) > tol)):
        n += 1
        ratio = fzt(z,dt)/dfdz(z)
        z = z - ratio
    
    yz = y(z)
    f = 1 - yz/r1_mag
    g = A*np.sqrt(yz/mu)
    gdot = 1 - yz/r2_mag
    
    v1 = 1/g*(r2 - f*r1)
    v2 = 1/g*(gdot*r2 - r1)
    
    return v1, v2

# Pork Chop Plots

In [ ]:
with open("/Users/thanegesite/Desktop/AE 502 (Eggl)/HW1/earth_ephem.csv") as file_name:
    file_read = csv.reader(file_name, quoting=csv.QUOTE_NONNUMERIC)
    earth = list(file_read)

In [ ]:
with open("/Users/thanegesite/Desktop/AE 502 (Eggl)/HW1/oum_ephem.csv") as file_name:
    file_read = csv.reader(file_name, quoting=csv.QUOTE_NONNUMERIC)
    oum = list(file_read)

In [98]:
with open("/Users/thanegesite/Desktop/AE 502 (Eggl)/HW1/bor_ephem.csv") as file_name:
    file_read = csv.reader(file_name, quoting=csv.QUOTE_NONNUMERIC)
    bor = list(file_read)

#### 'Oumouamoua

In [99]:
# range of days for departure and arrival
dep_days = 365 # 1 jan 2017 - 31 dec 2017
arr_days = 538 # 1 aug 2017 - 31 jan 2019

x_val = np.arange(0, dep_days, 1)
y_val = np.arange(0, arr_days, 1)

# earth ephemeride data
r1_initial = np.zeros((dep_days, 3))
v1_initial = np.zeros((dep_days, 3))

# load state vectors for every departure day
for i in range(dep_days):
    r1_initial[i] = earth[i][:3]
    v1_initial[i] = earth[i][3:]

# 'oumuamua ephemeride data
r2_initial = np.zeros((arr_days, 3))
v2_initial = np.zeros((arr_days, 3))

tof_array = np.zeros((arr_days, dep_days))
c3_array = np.zeros((arr_days, dep_days))
vinf_array = np.zeros((arr_days, dep_days))

# load state vectors for every arrival day
for i in range(arr_days):
    r2_initial[i] = oum[i][:3]
    v2_initial[i] = oum[i][3:]
    
for d in range(dep_days):
    for a in range(arr_days):
        tof = (a-d)*86400 # days to seconds
        delta_v = lambert_Curtis(r1_initial[d], r2_initial[a], tof, 'p')
        
        vinf_dep = np.linalg.norm([delta_v[0] - v1_initial[d]])
        vinf_arr = np.linalg.norm([delta_v[1] - v2_initial[a]])
        
        tof_array[d][a] = a-d
        c3_array[d][a] = vinf_dep**2
        vinf_array[d][a] = vinf_arr
        
#[X,Y] = np.meshgrid(x_val, y_val)
#fig, ax = plt.subplots(1, 1)

#Z = np.cos(X / 2) + np.sin(Y / 4)

#ax.contour(X, Y, Z)

#ax.set_title('Contour Plot')
#ax.set_xlabel('Days Since 1 Jan 2017')
#ax.set_ylabel('Days Since 1 Aug 2019')

#plt.show()

KeyboardInterrupt: 

#### Borisov

In [ ]:
# theoretically this would've followed the same procedure as above

# Convert Initial State Vectors to Suitable Orbital Elements

In [175]:
def orbital_elements(r, v, mu):
    # returns keplerian orbital elements
    # h: angular momentum vector
    # e: eccentricity
    # ra: right acension of the ascending node [radians]
    # i: inclination [radians]
    # omega: argument of perigee [radians]
    # f: true anomaly [radians]
    # a: semimajor axis [km]
    
    eps = 1e-10
    
    r_mag = np.linalg.norm(r)
    v_mag = np.linalg.norm(v)
    
    vr = np.dot(r,v)/r_mag
    
    h = np.cross(r,v)
    h_mag = np.linalg.norm(h)
    
    i = math.acos(h[2]/h_mag)
    
    n = np.cross(np.array([0,0,1]),h)
    n_mag = np.linalg.norm(n)
    
    if (n[1] >= 0):
        ra = math.acos(n[0]/n_mag)
    else:
        ra = 2*np.pi - math.acos(n[0]/n_mag)
    
    e = 1/mu*((v_mag**2 - mu/r_mag)*r - r_mag*vr*v) # eccentricity VECTOR
    e_mag = np.linalg.norm(e)
    
    if (e[2] >= 0):
        omega = math.acos(np.dot(n,e)/(n_mag*e_mag))
    else:
        omega = 2*np.pi - math.acos(np.dot(n,e)/(n_mag*e_mag))
    
    if (vr >= 0):
        f = math.acos(np.dot(e/e_mag, r/r_mag))
    else:
        f = 2*np.pi - math.acos(np.dot(e/e_mag, r/r_mag))
    
    a = h_mag**2/mu/(1-e_mag**2)
    
    return h_mag, e_mag, ra, i, omega, f, a

In [199]:
mu = 1.327e-11
r1i = np.array([3.515868886595499e-2, -3.162046390773074, 4.493983111703389])
v1i = np.array([2.317577766980901e-3, 9.843360903693031e-3, -1.541856855538041e-2])

r2i = np.array([7.249472033259724, 14.61063037906177, 14.24274452216359])
v2i = np.array([-8.241709369476881e-3, -1.156219024581502e-2, -1.317135977481448e-2],)

In [207]:
h1, e1, ra1, i1, omega1, f1, a1 = orbital_elements(r1i, v1i, mu)
h2, e2, ra2, i2, omega2, f2, a2 = orbital_elements(r2i, v2i, mu)

### Initial State Vectors in Keplerian Orbital Elements

In [211]:
print("angular momentum:", h1)
print("eccentricity:", e1)
print("right ascension:", np.degrees(ra1))
print("inclination:", np.degrees(i1))
print("argument of periapsis:", np.degrees(omega1))
print("true anomaly:", np.degrees(f1))
print("semi-major axis:", a1)

angular momentum: 0.014119936801866795
eccentricity: 19619963.95061179
right ascension: 157.5907125264039
inclination: 57.0768997643785
argument of periapsis: 185.00893364616311
true anomaly: 278.0105509050285
semi-major axis: -3.90299659840782e-08


In [212]:
print("angular momentum:", h2)
print("eccentricity:", e2)
print("right ascension:", np.degrees(ra2))
print("inclination:", np.degrees(i2))
print("argument of periapsis:", np.degrees(omega2))
print("true anomaly:", np.degrees(f2))
print("semi-major axis:", a2)

angular momentum: 0.0508897683513523
eccentricity: 74272840.67791073
right ascension: 308.2644450467314
inclination: 44.016596358788505
argument of periapsis: 191.841123237753
true anomaly: 276.9698403719356
semi-major axis: -3.537772943113907e-08


Interstellar objects, as denoted in the problem statement, have hyperbolic trajectories with respect to the Sun. As we can see above, the eccentricities are both much larger than 1, and the semi-major axes are both negative. These values both mathematically define the orbits of the two objects as hyperbolic. 

### Are these mission scenarios realistic?

Idk man are they realistic? Who do you think I am.